# Notebook for running the reconstructions.

- Contains climate reconstruction configurations
- Reconstruction config (cfg) is explained via comments (# ...)
- Actual reconstruction code outsourced to wrapper_new
- Test the reconstruction with a single montecarlo repetition first to estimate total run time
- If you do want to run longer reconstructions, I recommend exportint the code into a python file (see File -> Download as -> .py) and runit with nohup python script.py &

**Data**
- Check preprocessing notebooks if you want to use different data!


In [8]:
%%javascript 
Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('up');
Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('down');

<IPython.core.display.Javascript object>

In [7]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#SET PATH TO WHERE ALGORITHM PY FILES ARE STORED (wrapper_new.py)
import sys
sys.path.append('/home/mchoblet/paleoda/notebooks_misc/south_america/paper_github/algorithm')

In [18]:
import xarray as xr
from pathlib import Path
import numpy as np
import wrapper_new
import warnings
warnings.filterwarnings('ignore')
import tqdm
import os
import copy

#in case you want to modify wrapper, enable autoreload.
#also if you want to change kalman_filters.py and utils_new.py (loaded in wrapper_new.py), load them here to be sure that python files are reloaded (not sure if autoreload does that automagically)
#import utils_new
#import kalman_filters
#%load_ext autoreload
#%autoreload 2

## Example configuration (adjusted below)

In [11]:
cfg={
    'basepath':'/home/mchoblet/paleoda/data/model_data/', #folder with model data files (one for each variable)
        'vp':{ #vp is dictionary with <key>=variable name and <item> path to file (monthly resolution, with variable_name defined in key)
    'd18O':'ECHAM5_d18O_850_1849.nc',
    'prec':'ECHAM5_prec_850_1849.nc',
    'tsurf': 'ECHAM5_tsurf_850_1849.nc',
    'spei': 'ECHAM5_spei_851_1849.nc', #shorter by one year :-> limit all reconstructions to that time
    },
    'savepath':'/home/mchoblet/paleoda/results/experiments/paper/annual/normal/test.nc', #how to save reconstruction
    'multi_model_prior':None, #Either None (just use one model) or a list where first entry is dictionary with filepaths (recommend None (single model reconstructions) with posterior multi-model mean)
    'prior_cut_time':['0852'], #years where prior shall start (and end). 
    'avg':[12,1,2], #average that will be reconstruced. integer: starting month of annual mean, or list of integers for specific season mean
    'avg_proxies':4, #How to treat the annual proxies (from which month to start)
    'seed': 42, #random number seed (for monte carlo indices)
    'obsdata':[ #file paths for proxy record files. ORDER IMPORTANT! in all psm/time scale configs, the same order has to be applied.
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/trees_djf.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/corals_djf.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/glacier_ice.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/lakes.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/marine.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/sclero.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/speleos.nc',
        ],
    'regional_bounds':[[-65,35],[250,340]], #setting of a specific region if you only want to reconstruct that one. Longitudes given in 0 to 360 degrees. numbers from negative to positive. can also cross the 0 meridian. False or [[lat1,lat2],[lon1,lon2]]
    'only_regional_proxies':False, #if to limit proxies to the region defined in regional bounds (its better to look at your proxy files before)
    'proxy_time':['0001','2015'], #Cut proxy time series to these years
    'resample_mode': 'nearest', #temporal proxy interpolation linear/nearest (neighbor)
    'mask': 3, #windows size over which to mask actually non existing proxy values after the resampling
    'psm':['individual','individual','prec_weighted','individual','individual','individual', 'prec_weighted'], #psm config for each input file (individual -> proxy file contains regression parameters)
    'timescales':['1','5','10'], #time scales of multi-time scale reconstrution
    'db_timescales':['1','1','1','5','1','5','10'], #for each db define a timescale
    'smallest_timescale':1,
    'reuse':False, #if proxies should be reused also on larger time scales (will then also be resampled accordingly)
    'noise_assum':['None','None',0.5,0.5,0.5,0.5,0.5], #proxy noise for each db (None->part of proxy input file, from lin. regression, number: signal to noise ratio, can also be 'equal': then prior variance and proxy error set equal)
    'equal_fac':1, #option in case I use noise_assum='equal'. the proxy error is the prior variance times a factor
    ###KALMAN-FILTER
    'time': ['0001','2000'], #time of reconstruction (add leading zeroes for years prior to 1000!)
    'anomaly_time':['1750','1850'], #time over which to compute climate anomalies
    'reps':1, #number of monte carlo repetition
    'nens':100, #ensemble members
    'proxy_frac':None, #proxy fraction to use (None -> all proxy records, or number between 0 and 1)
}

# Wrapper of the reconstruction wrapper:

- as we run reconstruction for five different models, with different proxy input files and other parameter changes, I define a few helper dictionaries to make the task easier. One dictionary stores all model paths, another different model configurations

In [12]:
#model data folder
basepath3='/home/mchoblet/paleoda/data/model_data/upsampled/'

model_names={'CESM':
    {'tsurf':'CESM_tsurf_850_1850.nc',
     'prec':'CESM_prec_850_1850.nc',
     'd18O': 'CESM_d18O_850_1850.nc',
     'spei':'CESM_spei_pearson_12.nc',
    },
'isoGSM':
    {'tsurf':'CCSM_tsurf_851_1850.nc',
     'prec':'CCSM_prec_851_1850.nc',
     'd18O': 'CCSM_d18O_851_1850.nc',
     'spei':'CCSM_spei_pearson_12.nc',
    },
'iHadCM3':
    {'tsurf':'iHADCM3_tsurf_801_1952.nc',
     'prec':'iHADCM3_prec_801_1952.nc',
     'd18O': 'iHADCM3_d18O_801_1952.nc',
     'spei':'iHADCM3_spei_pearson_12.nc',
    },
'ECHAM5':
    {'tsurf':'ECHAM5_tsurf_850_1849.nc',
     'prec':'ECHAM5_prec_850_1849.nc',
    'd18O': 'ECHAM5_d18O_850_1849.nc',
     'spei':'ECHAM5_spei_pearson_12.nc',
    },
'GISS':
    {'tsurf':'GISS_tsurf_850_1849.nc',
     'prec':'GISS_prec_850_1849.nc',
   'd18O': 'GISS_d18O_850_1849.nc',
     'spei':'GISS_spei_pearson_12.nc',
    }
    }

#test that all files exist
for ii,k in model_names.items():
    for i,kk in k.items():
        for basepath in [basepath3]:
            try:
                xr.open_dataset(basepath+kk)
            except: print('Missing: ', kk,basepath)

In [13]:
#experiment configurations. different proxy intput files also require adjusting the psm configurations as I'm working with lists
#(at some point I might change this into something less error-prone, like another dictionary structure)

exp_config={
    'all_proxies':
        {
       'obsdata':[
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/trees_best_ann.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/corals_ann.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/marine.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/sclero.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/lakes.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/glacier_ice.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/speleos.nc',
        ],
        'psm':['individual','individual','individual','individual','individual','prec_weighted', 'prec_weighted'],
        'db_timescales':['1','1','1','5','5','1','10'],   
        'timescales':['1','5','10'],
        'noise_assum':['None','None',0.5,0.5,0.5,0.5,0.5],
        'proxy_frac_check':True,
        },
    'no_speleos':
        {
       'obsdata':[
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/trees_best_ann.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/corals_ann.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/marine.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/sclero.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/lakes.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/glacier_ice.nc',
        ],
        'psm':['individual','individual','individual','individual','individual','prec_weighted'],
        'db_timescales':['1','1','1','5','5','1',],   
        'timescales':['1','5'],
        'noise_assum':['None','None',0.5,0.5,0.5,0.5],
        'proxy_frac_check':False,
        },
    'no_trees':
        {
       'obsdata':[
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/corals_ann.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/marine.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/sclero.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/lakes.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/glacier_ice.nc',
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/speleos.nc',
        ],
        'psm':['individual','individual','individual','individual','prec_weighted', 'prec_weighted'],
        'db_timescales':['1','1','5','5','1','10'],   
        'timescales':['1','5','10'],
        'noise_assum':['None',0.5,0.5,0.5,0.5,0.5],
        'proxy_frac_check':False,
        },
    'speleos_only':
        {
       'obsdata':[
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/speleos.nc',
        ],
        'psm':['prec_weighted'],
        'db_timescales':['10'],   
        'timescales':['1','5','10'],
        'noise_assum':[0.5],
        'proxy_frac_check':False,
        },
    'trees_only':
        {
       'obsdata':[
        '/home/mchoblet/paleoda/data/proxy_dbs/paper/trees_best_ann.nc',
        ],
        'psm':['individual'],
        'db_timescales':['1'],   
        'timescales':['1'],
        'noise_assum':['None'],
        'proxy_frac_check':False,
        },
          }

# Reconstructions:

- 50 Repetitions with 100 ensemble members
- Annual/DJF
- 80% of proxy records
- different experiments: allproxies, speleos_only, trees_only, no_speleos, no_trees
- Usual SNR=0.5 / Equal Error reconstruction (third option could be 0.5 * model variance to give more weight to proxies)

Each reconstruction takes ~0.5GB

**Nested file structure for saving reconstructions. Put all models for one configuration into a folder. That will simplify postprocessing**

* errory_type (normal, equal variance)
    * annual/djf season reconstruction
        * experiment_configuration (e.g. all proxy records, only trees etc)
        
Folders and parents are created on the fly using pathlib.Path

In [14]:
#Usual error calculation (calibration error + snr)

cfg['avg_proxies']=4
folder='/home/mchoblet/paleoda/results/experiments/paper/final_new/normal_err/'

for k,avg in {'annual':4,'djf':[12,1,2]}.items():
    cfg['avg']=avg
    for kk,pf in {'proxy_frac':0.8}.items(): #'proxy_all':None,
        #print(pf)
        cfg['proxy_frac']=pf
        #loop for experiments
        for kkk,c in exp_config.items():
            for name,conf in c.items():
                cfg[name]=conf
            #echange ann/djf path if cfg['obs_data']
            for i_p,proxypath in enumerate(cfg['obsdata']):
                if k=='djf':  cfg['obsdata'][i_p]=proxypath.replace('_ann.nc','_djf.nc')
                else:  cfg['obsdata'][i_p]=proxypath.replace('_djf.nc','_ann.nc')
            #loop over models
            for mods, modpaths in model_names.items():
                cfg['vp']=modpaths
                cfg['savepath']=folder+k+'/'+kk+'/'+kkk+'/'+mods+'.nc' 
                dir_name=os.path.dirname(cfg['savepath'])
                Path(dir_name).mkdir(parents=True, exist_ok=True)
                print('PaleoDA for',mods, 'in config:',k,'|',kk,'|',kkk)
                out=wrapper_new.wrapper_paper(cfg)

PaleoDA for CESM in config: annual | proxy_frac | all_proxies
PaleoDA for isoGSM in config: annual | proxy_frac | all_proxies
PaleoDA for iHadCM3 in config: annual | proxy_frac | all_proxies
PaleoDA for ECHAM5 in config: annual | proxy_frac | all_proxies
PaleoDA for GISS in config: annual | proxy_frac | all_proxies
PaleoDA for CESM in config: annual | proxy_frac | no_speleos
PaleoDA for isoGSM in config: annual | proxy_frac | no_speleos
PaleoDA for iHadCM3 in config: annual | proxy_frac | no_speleos
PaleoDA for ECHAM5 in config: annual | proxy_frac | no_speleos
PaleoDA for GISS in config: annual | proxy_frac | no_speleos
PaleoDA for CESM in config: annual | proxy_frac | no_trees
PaleoDA for isoGSM in config: annual | proxy_frac | no_trees
PaleoDA for iHadCM3 in config: annual | proxy_frac | no_trees
PaleoDA for ECHAM5 in config: annual | proxy_frac | no_trees
PaleoDA for GISS in config: annual | proxy_frac | no_trees
PaleoDA for CESM in config: annual | proxy_frac | speleos_only
PaleoD

In [ ]:
cfg['avg_proxies']=4
folder='/home/mchoblet/paleoda/results/experiments/paper/final_new/same_err/'
cfg['proxy_fac']=1

for k,avg in {'annual':4,'djf':[12,1,2]}.items():
    cfg['avg']=avg
    #for kk,pf in {'proxy_all':None,'proxy_frac':0.8}.items():
    for kk,pf in {'proxy_frac':0.8}.items(): #'proxy_all':None,
        #print(pf)
        cfg['proxy_frac']=pf
        #loop for experiments
        for kkk,c in exp_config.items():
            for name,conf in c.items():
                cfg[name]=conf
            #replace noise which is a number by 'equal'. proxy_fac is set to 1
            for i_n, noise in enumerate(cfg['noise_assum']):
                if isinstance(noise,int) or isinstance(noise,float):
                    cfg['noise_assum'][i_n]='equal'
            #echange ann/djf path if cfg['obs_data']
            for i_p,proxypath in enumerate(cfg['obsdata']):
                if k=='djf': cfg['obsdata'][i_p]=proxypath.replace('_ann.nc','_djf.nc')
                else: cfg['obsdata'][i_p]=proxypath.replace('_djf.nc','_ann.nc')
            #loop over models
            for mods, modpaths in model_names.items():
                cfg['vp']=modpaths
                cfg['savepath']=folder+k+'/'+kkk+'/'+mods+'.nc'
                dir_name=os.path.dirname(cfg['savepath'])
                Path(dir_name).mkdir(parents=True, exist_ok=True)
                b=wrapper_new.wrapper_paper(cfg)
                

In [ ]:
#reduce proxy error to half the prior variance

cfg['avg_proxies']=4
folder='/home/mchoblet/paleoda/results/experiments/paper/final_new/half_err/'
cfg['proxy_fac']=0.5

for k,avg in {'annual':4,'djf':[12,1,2]}.items():
    cfg['avg']=avg
    #for kk,pf in {'proxy_all':None,'proxy_frac':0.8}.items():
    for kk,pf in {'proxy_frac':0.8}.items(): #'proxy_all':None,
        #print(pf)
        cfg['proxy_frac']=pf
        #loop for experiments
        for kkk,c in exp_config.items():
            for name,conf in c.items():
                cfg[name]=conf
            
            #replace noise which is a number by 'equal'. proxy_fac is set to 1
            for i_n, noise in enumerate(cfg['noise_assum']):
                if isinstance(noise,int) or isinstance(noise,float):
                    cfg['noise_assum'][i_n]='equal'
                
            #echange ann/djf path if cfg['obs_data']
            for i_p,proxypath in enumerate(cfg['obsdata']):
                if k=='djf': cfg['obsdata'][i_p]=proxypath.replace('_ann.nc','_djf.nc')
                else: cfg['obsdata'][i_p]=proxypath.replace('_djf.nc','_ann.nc')
            #loop over models
            for mods, modpaths in model_names.items():
                cfg['vp']=modpaths
                cfg['savepath']=folder+k+'/'+kkk+'/'+mods+'_'+'.nc'
                dir_name=os.path.dirname(cfg['savepath'])
                Path(dir_name).mkdir(parents=True, exist_ok=True)
                wrapper_new.wrapper_paper(cfg)